## GPU Puzzles in CUDA C++
By Devin Shah - [@devinshah16](https://twitter.com/DevinShah16)

Puzzles adapted from [Sasha Rush](http://rush-nlp.com/)

GPUs are pretty cool.

Make your own copy of this notebook in Colab, turn on GPU mode in the settings (`Runtime / Change runtime type`, then set `Hardware accelerator` to `GPU`), and
then get to coding. ***You might get a warning saying that the GPU is not being used, but it is in fact being used. Ignore this warning. If using a free version, be careful of quotas.***


Read the [CUDA C++ bindings guide ](https://docs.nvidia.com/cuda/pdf/CUDA_C_Programming_Guide.pdf)

In [ ]:
!git clone https://github.com/AIS-UCLA/cuda-puzzles

In [ ]:
%cd cuda-puzzles/GPU_puzzlers_exec

Make sure `nvcc` is installed. If it is not, this notebook will not work.

In [ ]:
!nvcc --version

## Puzzle 1 - Vector Add
Parallelize this CPU program
```
for (int i = 0; i < N; i++){
  C[i] = A[i] + B[i]
}
```
Implement a kernel that adds together each position of `A` and `B` and stores it in `C`. You have 1 thread per position.

In [ ]:
%%writefile zip_kernel.cu
#include <cuda_runtime.h>

__global__ void VecAdd(float* A, float* B, float* C) {

}

In [ ]:
!nvcc -o zip -arch=sm_75 zip_runner.cu zip_kernel.cu
!./zip
!compute-sanitizer ./zip

## Puzzle 2 - Vector Add with Guards
Parallelize this CPU program
```
for (int i = 0; i < N; i++){
  C[i] = A[i] + B[i]
}
```
The number of threads is now greater than `N`

In [ ]:
%%writefile zip_guards_kernel.cu
#include <cuda_runtime.h>

__global__ void VecAdd_Guards(float* A, float* B, float* C, int N){

}

In [ ]:
!nvcc -o zip_guards -arch=sm_75 zip_runner_guards.cu zip_guards_kernel.cu
!./zip_guards
!compute-sanitizer ./zip_guards

## Puzzle 3 - Broadcast

Implement a kernel that adds `A` and `B` and stores it in `C`.
Inputs `A` and `B` are vectors. You have more threads than positions.
1D indexing doesn't work for 2D arrays in CUDA C++. You can calculate the index from i and j by computing `i * size + j`. There is only 1 thread block

In [ ]:
%%writefile broadcast_kernel.cu
#include <cuda_runtime.h>

__global__ void Broadcast(float* A, float* B, float* C, int size) {

}

In [ ]:
!nvcc -o broadcast -arch=sm_75 broadcast_runner.cu broadcast_kernel.cu
!./broadcast
!compute-sanitizer ./broadcast

## Puzzle 4 - Blocks

Implement a kernel that adds 10 to each position of `A` and stores it in `C`.
You have fewer threads per block than the size of `A`. (i.e. will need to use multiple blocks to cover the entire size)

Use `blockIdx * blockDim + threadIdx`

*Tip: A block is a group of threads. The number of threads per block is limited, but we can
have many different blocks. Variable `cuda.blockIdx` tells us what block we are in.*

In [ ]:
%%writefile blocks_kernel.cu
#include <cuda_runtime.h>

__global__ void Blocks(float* A, float* C, float size) {

}

In [ ]:
!nvcc -o blocks -arch=sm_75 blocks_runner.cu blocks_kernel.cu
!./blocks
!compute-sanitizer ./blocks

## Puzzle 5 - Blocks 2D

Implement the same kernel in 2D.  You have fewer threads per block
than the size of `A` in both directions.
A is 2D and C is now 2D, will need threads along `.x` and `.y` now

In [ ]:
%%writefile map2d_block_kernel.cu
#include <cuda_runtime.h>

__global__ void Map2DBlock(float* A, float* C, int size) {

}

In [ ]:
!nvcc -o map2d_block -arch=sm_75 map2d_block_runner.cu map2d_block_kernel.cu
!./map2d_block
!compute-sanitizer ./map2d_block

## Puzzle 6 - Transpose

Implement a kernel that transposes `in` (2D matrix) and writes to `out` 
`Tip` How do you write index via row-major order vs col-major order?


`in` is of shape `MxN`, `out` is of shape `NxM`

In [ ]:
%%writefile transpose_kernel.cu
#include <cuda_runtime.h>

__global__ void Transpose(float *in, float *out, int M, int N){

}

In [ ]:
!nvcc -o transpose -arch=sm_75 transpose_runner.cu transpose_kernel.cu
!./transpose
!compute-sanitizer ./transpose

## Puzzle 7 - Transpose Shared

Implement a kernel that transposes `in` (2D matrix) and writes to `out`

`Tip 1` Think about the size of your shared memory array, and how you would index it differently from in/out which of `larger` shape

`Tip 2` We need to do transposing at 2 levels (within the blocks, shared memory) and across blocks (may need to swap blockIdx)

`in` is of shape `MxN`, `out` is of shape `NxM`


In [ ]:
%%writefile transpose_shared_kernel.cu
#include <cuda_runtime.h>

__global__ void _transpose_shared(float *in, float *out, int M, int N){
  extern __shared__ float arr_shared[];

}

In [ ]:
!nvcc -o transpose_shared -arch=sm_75 transpose_shared_runner.cu transpose_shared_kernel.cu
!./transpose_shared
!compute-sanitizer ./transpose_shared

## Puzzle 8 - Matrix Multiplication
A matmul on CPU can be written as a 3 nested for loop program.
```
for (int i = 0; i < M; i++){
  for (int j = 0; j < N; j++){
    for (int k = 0; k < K; k++){
      C[i][j] += A[i*K+k] * B[k*N+j]; // C[i][j] += A[i][k] * B[k][j]
    }
  }
}
```

Convert this implementation to CUDA (think about about what axis would be parallelized)

`A` is of shape `MxK`, B is of shape `KxN`

For this problem `M=N=K=2048`, and the size wil evenly divide by the thread block size of 16

In [ ]:
%%writefile matmul_kernel.cu
#include <cuda_runtime.h>

__global__ void matmul(float *A, float *B, float *C, int M, int N, int K){

}

In [ ]:
!nvcc -o matmul -arch=sm_75 matmul_runner.cu matmul_kernel.cu
!./matmul
!compute-sanitizer ./matmul

## Puzzle 9 - Matrix Multiplication with Guards
A matmul on CPU can be written as a 3 nested for loop program.
```
for (int i = 0; i < M; i++){
  for (int j = 0; j < N; j++){
    for (int k = 0; k < K; k++){
      C[i][j] += A[i*K+k] * B[k*N+j]; // C[i][j] += A[i][k] * B[k][j]
    }
  }
}
```

Convert this implementation to CUDA (think about about what axis would be parallelized)

`A` is of shape `MxK`, B is of shape `KxN`

For this problem `M=1025, N=2048, K = 2048`, and the size will need to round up to spawn enough thread blocks along M (look at matmul_runner.cu)

In [ ]:
%%writefile matmul_guards_kernel.cu
#include <cuda_runtime.h>

__global__ void matmul_guards(float *A, float *B, float *C, int M, int N, int K){

}

In [ ]:
!nvcc -o matmul_guards -arch=sm_75 matmul_guards_runner.cu matmul_guards_kernel.cu
!./matmul_guards
!compute-sanitizer ./matmul_guards

## Puzzle 10 - Coalesced Matrix Multiplication
A matmul on CPU can be written as a 3 nested for loop program.
```
for (int i = 0; i < M; i++){
  for (int j = 0; j < N; j++){
    for (int k = 0; k < K; k++){
      C[i][j] += A[i*K+k] * B[k*N+j]; // C[i][j] += A[i][k] * B[k][j]
    }
  }
}
```
Given that

`int i = blockIdx.x * blockDim.x + threadIdx.x;`

`int j = blockIdx.y * blockDim.y + threadIdx.y;`

Does `C[i][j] += A[i*K+k] * B[k*N+j]` achieve coalescing?

`Tip`: Remember we always want to map `threadIdx.x` along the contiguous dimension, also look at matmul_coalseced_runner.cu to see the gridDim and blockDim

`A` is of shape `MxK`, B is of shape `KxN`

For this problem `M=1025, N=2048, K = 2048`

In [ ]:
%%writefile matmul_coalesced_kernel.cu
#include <cuda_runtime.h>

__global__ void matmul_coalesced(float *A, float *B, float *C, int M, int N, int K){

}

In [ ]:
!nvcc -o matmul_coaleseced -arch=sm_75 matmul_coalesced_runner.cu matmul_coalesced_kernel.cu
!./matmul_coaleseced
!compute-sanitizer ./matmul_coaleseced

## Puzzle 11 - Matrix Multiplication w/ Shared Memory
A matmul on CPU can be written as a 3 nested for loop program.
```
for (int i = 0; i < M; i++){
  for (int j = 0; j < N; j++){
    for (int k = 0; k < K; k++){
      C[i][j] += A[i*K+k] * B[k*N+j]; // C[i][j] += A[i][k] * B[k][j]
    }
  }
}
```

Speedup your previous matmul implementations w/ shared memory

In [ ]:
%%writefile matmul_shared_kernel.cu
#include <cuda_runtime.h>

__global__ void matmul_shared(float *A, float *B, float *C, int M, int N, int K, int BLOCK_M, int BLOCK_N, int BLOCK_K){
  extern __shared__ float shared[];
  float *As = shared;
  float *Bs = shared + BLOCK_M*BLOCK_K;

  int j = blockIdx.x * blockDim.x + threadIdx.x;
  int i = blockIdx.y * blockDim.y + threadIdx.y;

  float inner_prod = 0.0f;
  for (int k_tile = 0; k_tile < K/BLOCK_K; k_tile++) {
    // .y threads are mapped along M dim
    // .x threads are mapped along N dim
    int a_col = (k_tile*BLOCK_K + threadIdx.x);
    int b_row = (k_tile*BLOCK_K + threadIdx.y);
    As[threadIdx.y*BLOCK_K+threadIdx.x] = A[i*K + a_col];
    Bs[threadIdx.y*BLOCK_N+threadIdx.x] = B[b_row*N + j];
    __syncthreads();
    for (int kk = 0; kk < BLOCK_K; kk++){
      inner_prod += As[threadIdx.y*BLOCK_K+kk] * Bs[kk*BLOCK_K+threadIdx.x];
    }
    __syncthreads();
  }

  // 1 thread per output element
  if (i < M && j < N){
    C[i*N+j] = inner_prod;
  }

In [ ]:
!nvcc -o matmul_shared -arch=sm_75 matmul_shared_runner.cu matmul_shared_kernel.cu
!./matmul_shared
!compute-sanitizer ./matmul_shared

## Puzzle 12 - 1D Indexing

Copy a 1D tensor using CUDA threads.

**Signature:** `Index1D(float* in, float* out, int d)`

- Shape: `(d,)` where d=64
- Each thread copies one element
- Use `threadIdx.x` to index

**Launch:** `<<<1, d>>>`

In [ ]:
%%writefile index1d_kernel.cu
#include <cuda_runtime.h>

__global__ void Index1D(float* in, float* out, int d) {
    // TODO: use threadIdx.x to copy elements
}


In [ ]:
%%writefile index1d_kernel.cu
#include <cuda_runtime.h>

__global__ void Index1D(float* in, float* out, int d) {
    // TODO: use threadIdx.x to copy elements
}


## Puzzle 13 - 2D Indexing

Copy a 2D tensor (n,d) using 2D thread blocks.

**Signature:** `Index2D(float* in, float* out, int n, int d)`

- Shape: `(n, d)` where n=16, d=64
- `threadIdx.y` → rows (n)
- `threadIdx.x` → cols (d)
- Row-major index: `i*d + j`

**Launch:** `<<<1, dim3(d, n)>>>`

In [ ]:
%%writefile index2d_kernel.cu
#include <cuda_runtime.h>

__global__ void Index2D(float* in, float* out, int n, int d) {
    // TODO: use threadIdx.y, threadIdx.x
    // Remember: i*d + j for row-major indexing
}


In [ ]:
%%writefile index2d_kernel.cu
#include <cuda_runtime.h>

__global__ void Index2D(float* in, float* out, int n, int d) {
    // TODO: use threadIdx.y, threadIdx.x
    // Remember: i*d + j for row-major indexing
}


## Puzzle 14 - 3D Indexing

Copy a 3D tensor (h,n,d) using blocks + 2D threads.

**Signature:** `Index3D(float* in, float* out, int h, int n, int d)`

- Shape: `(h, n, d)` where h=20, n=16, d=64
- `blockIdx.x` → height (h)
- `threadIdx.y` → rows (n)
- `threadIdx.x` → cols (d)
- Index: `i*n*d + j*d + k`

**Launch:** `<<<h, dim3(d, n)>>>`

In [ ]:
%%writefile index3d_kernel.cu
#include <cuda_runtime.h>

__global__ void Index3D(float* in, float* out, int h, int n, int d) {
    // TODO: use blockIdx.x, threadIdx.y, threadIdx.x
    // Formula: i*n*d + j*d + k
}


In [ ]:
%%writefile index3d_kernel.cu
#include <cuda_runtime.h>

__global__ void Index3D(float* in, float* out, int h, int n, int d) {
    // TODO: use blockIdx.x, threadIdx.y, threadIdx.x
    // Formula: i*n*d + j*d + k
}


## Puzzle 15 - 4D Indexing

Copy a 4D tensor (b,h,n,d) - like transformers!

**Signature:** `Index4D(float* in, float* out, int b, int h, int n, int d)`

- Shape: `(b, h, n, d)` where b=4, h=20, n=16, d=64
- `blockIdx.y` → batch (b)
- `blockIdx.x` → heads (h)
- `threadIdx.y` → seq_len (n)
- `threadIdx.x` → dim (d)
- Index: `i*h*n*d + j*n*d + k*d + l`

**Launch:** `<<<dim3(h, b), dim3(d, n)>>>`

In [ ]:
%%writefile index4d_kernel.cu
#include <cuda_runtime.h>

__global__ void Index4D(float* in, float* out, int b, int h, int n, int d) {
    // TODO: use blockIdx.y, blockIdx.x, threadIdx.y, threadIdx.x
    // Formula: i*h*n*d + j*n*d + k*d + l
}


In [ ]:
%%writefile index4d_kernel.cu
#include <cuda_runtime.h>

__global__ void Index4D(float* in, float* out, int b, int h, int n, int d) {
    // TODO: use blockIdx.y, blockIdx.x, threadIdx.y, threadIdx.x
    // Formula: i*h*n*d + j*n*d + k*d + l
}
